In [1]:
import nibabel as nib
import matplotlib.pyplot as plt
from scipy import ndimage
import cv2
import glob
import nilearn
from nilearn import image
import numpy as np
from PIL import Image
import pandas as pd
import matplotlib

In [2]:
train1=pd.read_csv('dti_train_fold1.csv', index_col=0)
train1.groupby('Label').count()


,DWI_file,day
Label,,
AD,37,37
MCI,4,4
Normal,193,193


In [24]:
train_labels=[]
train_file_names=[]
dftrain=train1
for i in range(len(dftrain['DWI_file'])):
    file_name=dftrain.iloc[i].DWI_file
    label=dftrain.iloc[i].Label

    mri = nib.load('/raid/aidana_massalimova/nii_data/FA/'+file_name)
    zooms = mri.header.get_zooms()
    old_affine = mri.affine
    new_affine = nib.affines.rescale_affine(affine=old_affine, shape=mri.get_fdata().shape, zooms=zooms, new_shape=(96,96,96))
    new_mri = image.resample_img(mri, target_affine=new_affine,target_shape=(96,96,96), interpolation='nearest')    
    epi_img_data = new_mri.get_fdata()
    if label=='Normal':
        slice_0 = epi_img_data[96//2, :, :]
        slice_1 = epi_img_data[:, 96//2, :]
        slice_2 = epi_img_data[:, :, 96//2]

        slice_0 /= slice_0.max()/255.0
        slice_1 /= slice_1.max()/255.0
        slice_2 /= slice_2.max()/255.0

        fft_p0 = slice_0.astype(np.uint8)
        fft_p1 = slice_1.astype(np.uint8)
        fft_p2 = slice_2.astype(np.uint8)
        mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
        mri = Image.fromarray(mri)
        name=file_name[0:-4]+'_1.png'
        train_labels.append(label)
        train_file_names.append(name)
        mri.save('fold1/train/{}'.format(name))                        
    elif label=='MCI':
        for i in range(24,72):
            slice_0 = epi_img_data[i, :, :]
            slice_1 = epi_img_data[:, i, :]
            slice_2 = epi_img_data[:, :, i]

            slice_0 /= slice_0.max()/255.0
            slice_1 /= slice_1.max()/255.0
            slice_2 /= slice_2.max()/255.0

            fft_p0 = slice_0.astype(np.uint8)
            fft_p1 = slice_1.astype(np.uint8)
            fft_p2 = slice_2.astype(np.uint8)
            mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
            mri = Image.fromarray(mri)
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
            mri.save('fold1/train/{}'.format(name))        
 

    elif label=='AD':
        for i in range(46,51):
            slice_0 = epi_img_data[i, :, :]
            slice_1 = epi_img_data[:, i, :]
            slice_2 = epi_img_data[:, :, i]

            slice_0 /= slice_0.max()/255.0
            slice_1 /= slice_1.max()/255.0
            slice_2 /= slice_2.max()/255.0

            fft_p0 = slice_0.astype(np.uint8)
            fft_p1 = slice_1.astype(np.uint8)
            fft_p2 = slice_2.astype(np.uint8)

            mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
            mri = Image.fromarray(mri)
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
            mri.save('fold1/train/{}'.format(name))    
            

In [25]:
fold1_train={'File_name':train_file_names,
      'Label':train_labels}  
df_fold1_train= pd.DataFrame (fold1_train, columns =  ['File_name','Label'])
df_fold1_train.groupby('Label').count()
df_fold1_train
from sklearn.utils import shuffle
df_fold1_train = shuffle(df_fold1_train)
np.savetxt('train_fold1.txt', df_fold1_train.values,delimiter=" ", fmt="%s")
df_fold1_train.groupby('Label').count()

,File_name
Label,
AD,185
MCI,192
Normal,193


In [26]:
train2=pd.read_csv('dti_train_fold2.csv', index_col=0)
train2.groupby('Label').count()
train2

,DWI_file,day,Label
Subject,,,
OAS30076,FA_sub-OAS30076_ses-d1735_run-02_dwi.nii,1735,AD
OAS30085,FA_sub-OAS30085_ses-d1566_run-02_dwi.nii,1566,AD
OAS30136,FA_sub-OAS30136_ses-d0063_run-02_dwi.nii,63,AD
OAS30139,FA_sub-OAS30139_ses-d0101_run-02_dwi.nii,101,AD
OAS30206,FA_sub-OAS30206_ses-d2303_run-02_dwi.nii,2303,AD
...,...,...,...
OAS30402,FA_sub-OAS30402_ses-d0015_run-02_dwi.nii,15,Normal
OAS30403,FA_sub-OAS30403_ses-d1232_run-02_dwi.nii,1232,Normal
OAS30405,FA_sub-OAS30405_ses-d1883_run-02_dwi.nii,1883,Normal


In [27]:
train_labels=[]
train_file_names=[]
dftrain=train2
for i in range(len(dftrain['DWI_file'])):
    file_name=dftrain.iloc[i].DWI_file
    label=dftrain.iloc[i].Label

    mri = nib.load('/raid/aidana_massalimova/nii_data/FA/'+file_name)
    zooms = mri.header.get_zooms()
    old_affine = mri.affine
    new_affine = nib.affines.rescale_affine(affine=old_affine, shape=mri.get_fdata().shape, zooms=zooms, new_shape=(96,96,96))
    new_mri = image.resample_img(mri, target_affine=new_affine,target_shape=(96,96,96), interpolation='nearest')    
    epi_img_data = new_mri.get_fdata()
    if label=='Normal':
        slice_0 = epi_img_data[96//2, :, :]
        slice_1 = epi_img_data[:, 96//2, :]
        slice_2 = epi_img_data[:, :, 96//2]

        slice_0 /= slice_0.max()/255.0
        slice_1 /= slice_1.max()/255.0
        slice_2 /= slice_2.max()/255.0

        fft_p0 = slice_0.astype(np.uint8)
        fft_p1 = slice_1.astype(np.uint8)
        fft_p2 = slice_2.astype(np.uint8)
        mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
        mri = Image.fromarray(mri)
        name=file_name[0:-4]+'_1.png'
        train_labels.append(label)
        train_file_names.append(name)
        mri.save('fold2/train/{}'.format(name))                        
    elif label=='MCI':
        for i in range(24,72):
            slice_0 = epi_img_data[i, :, :]
            slice_1 = epi_img_data[:, i, :]
            slice_2 = epi_img_data[:, :, i]

            slice_0 /= slice_0.max()/255.0
            slice_1 /= slice_1.max()/255.0
            slice_2 /= slice_2.max()/255.0

            fft_p0 = slice_0.astype(np.uint8)
            fft_p1 = slice_1.astype(np.uint8)
            fft_p2 = slice_2.astype(np.uint8)
            mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
            mri = Image.fromarray(mri)
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
            mri.save('fold2/train/{}'.format(name))        
 

    elif label=='AD':
        for i in range(46,51):
            slice_0 = epi_img_data[i, :, :]
            slice_1 = epi_img_data[:, i, :]
            slice_2 = epi_img_data[:, :, i]

            slice_0 /= slice_0.max()/255.0
            slice_1 /= slice_1.max()/255.0
            slice_2 /= slice_2.max()/255.0

            fft_p0 = slice_0.astype(np.uint8)
            fft_p1 = slice_1.astype(np.uint8)
            fft_p2 = slice_2.astype(np.uint8)

            mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
            mri = Image.fromarray(mri)
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
            mri.save('fold2/train/{}'.format(name))    
            

In [28]:
fold1_train={'File_name':train_file_names,
      'Label':train_labels}  
df_fold1_train= pd.DataFrame (fold1_train, columns =  ['File_name','Label'])
df_fold1_train.groupby('Label').count()
df_fold1_train
from sklearn.utils import shuffle
df_fold1_train = shuffle(df_fold1_train)
np.savetxt('train_fold2.txt', df_fold1_train.values,delimiter=" ", fmt="%s")
df_fold1_train.groupby('Label').count()

,File_name
Label,
AD,185
MCI,192
Normal,193


In [29]:
train3=pd.read_csv('dti_train_fold3.csv', index_col=0)
train3.groupby('Label').count()
train3

,DWI_file,day,Label
Subject,,,
OAS30076,FA_sub-OAS30076_ses-d1735_run-02_dwi.nii,1735,AD
OAS30085,FA_sub-OAS30085_ses-d1566_run-02_dwi.nii,1566,AD
OAS30136,FA_sub-OAS30136_ses-d0063_run-02_dwi.nii,63,AD
OAS30139,FA_sub-OAS30139_ses-d0101_run-02_dwi.nii,101,AD
OAS30206,FA_sub-OAS30206_ses-d2303_run-02_dwi.nii,2303,AD
...,...,...,...
OAS30402,FA_sub-OAS30402_ses-d0015_run-02_dwi.nii,15,Normal
OAS30403,FA_sub-OAS30403_ses-d1232_run-02_dwi.nii,1232,Normal
OAS30405,FA_sub-OAS30405_ses-d1883_run-02_dwi.nii,1883,Normal


In [30]:
train_labels=[]
train_file_names=[]
dftrain=train3
for i in range(len(dftrain['DWI_file'])):
    file_name=dftrain.iloc[i].DWI_file
    label=dftrain.iloc[i].Label

    mri = nib.load('/raid/aidana_massalimova/nii_data/FA/'+file_name)
    zooms = mri.header.get_zooms()
    old_affine = mri.affine
    new_affine = nib.affines.rescale_affine(affine=old_affine, shape=mri.get_fdata().shape, zooms=zooms, new_shape=(96,96,96))
    new_mri = image.resample_img(mri, target_affine=new_affine,target_shape=(96,96,96), interpolation='nearest')    
    epi_img_data = new_mri.get_fdata()
    if label=='Normal':
        slice_0 = epi_img_data[96//2, :, :]
        slice_1 = epi_img_data[:, 96//2, :]
        slice_2 = epi_img_data[:, :, 96//2]

        slice_0 /= slice_0.max()/255.0
        slice_1 /= slice_1.max()/255.0
        slice_2 /= slice_2.max()/255.0

        fft_p0 = slice_0.astype(np.uint8)
        fft_p1 = slice_1.astype(np.uint8)
        fft_p2 = slice_2.astype(np.uint8)
        mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
        mri = Image.fromarray(mri)
        name=file_name[0:-4]+'_1.png'
        train_labels.append(label)
        train_file_names.append(name)
        mri.save('fold3/train/{}'.format(name))                        
    elif label=='MCI':
        for i in range(24,72):
            slice_0 = epi_img_data[i, :, :]
            slice_1 = epi_img_data[:, i, :]
            slice_2 = epi_img_data[:, :, i]

            slice_0 /= slice_0.max()/255.0
            slice_1 /= slice_1.max()/255.0
            slice_2 /= slice_2.max()/255.0

            fft_p0 = slice_0.astype(np.uint8)
            fft_p1 = slice_1.astype(np.uint8)
            fft_p2 = slice_2.astype(np.uint8)
            mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
            mri = Image.fromarray(mri)
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
            mri.save('fold3/train/{}'.format(name))        
 

    elif label=='AD':
        for i in range(46,51):
            slice_0 = epi_img_data[i, :, :]
            slice_1 = epi_img_data[:, i, :]
            slice_2 = epi_img_data[:, :, i]

            slice_0 /= slice_0.max()/255.0
            slice_1 /= slice_1.max()/255.0
            slice_2 /= slice_2.max()/255.0

            fft_p0 = slice_0.astype(np.uint8)
            fft_p1 = slice_1.astype(np.uint8)
            fft_p2 = slice_2.astype(np.uint8)

            mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
            mri = Image.fromarray(mri)
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
            mri.save('fold3/train/{}'.format(name))    
            

In [31]:
fold1_train={'File_name':train_file_names,
      'Label':train_labels}  
df_fold1_train= pd.DataFrame (fold1_train, columns =  ['File_name','Label'])
df_fold1_train.groupby('Label').count()
df_fold1_train
from sklearn.utils import shuffle
df_fold1_train = shuffle(df_fold1_train)
np.savetxt('train_fold3.txt', df_fold1_train.values,delimiter=" ", fmt="%s")
df_fold1_train.groupby('Label').count()

,File_name
Label,
AD,190
MCI,192
Normal,192


In [32]:
train4=pd.read_csv('dti_train_fold4.csv', index_col=0)
train4.groupby('Label').count()
train4

,DWI_file,day,Label
Subject,,,
OAS30076,FA_sub-OAS30076_ses-d1735_run-02_dwi.nii,1735,AD
OAS30085,FA_sub-OAS30085_ses-d1566_run-02_dwi.nii,1566,AD
OAS30136,FA_sub-OAS30136_ses-d0063_run-02_dwi.nii,63,AD
OAS30139,FA_sub-OAS30139_ses-d0101_run-02_dwi.nii,101,AD
OAS30206,FA_sub-OAS30206_ses-d2303_run-02_dwi.nii,2303,AD
...,...,...,...
OAS30402,FA_sub-OAS30402_ses-d0015_run-02_dwi.nii,15,Normal
OAS30403,FA_sub-OAS30403_ses-d1232_run-02_dwi.nii,1232,Normal
OAS30405,FA_sub-OAS30405_ses-d1883_run-02_dwi.nii,1883,Normal


In [33]:
train_labels=[]
train_file_names=[]
dftrain=train4
for i in range(len(dftrain['DWI_file'])):
    file_name=dftrain.iloc[i].DWI_file
    label=dftrain.iloc[i].Label

    mri = nib.load('/raid/aidana_massalimova/nii_data/FA/'+file_name)
    zooms = mri.header.get_zooms()
    old_affine = mri.affine
    new_affine = nib.affines.rescale_affine(affine=old_affine, shape=mri.get_fdata().shape, zooms=zooms, new_shape=(96,96,96))
    new_mri = image.resample_img(mri, target_affine=new_affine,target_shape=(96,96,96), interpolation='nearest')    
    epi_img_data = new_mri.get_fdata()
    if label=='Normal':
        slice_0 = epi_img_data[96//2, :, :]
        slice_1 = epi_img_data[:, 96//2, :]
        slice_2 = epi_img_data[:, :, 96//2]

        slice_0 /= slice_0.max()/255.0
        slice_1 /= slice_1.max()/255.0
        slice_2 /= slice_2.max()/255.0

        fft_p0 = slice_0.astype(np.uint8)
        fft_p1 = slice_1.astype(np.uint8)
        fft_p2 = slice_2.astype(np.uint8)
        mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
        mri = Image.fromarray(mri)
        name=file_name[0:-4]+'_1.png'
        train_labels.append(label)
        train_file_names.append(name)
        mri.save('fold4/train/{}'.format(name))                        
    elif label=='MCI':
        for i in range(24,72):
            slice_0 = epi_img_data[i, :, :]
            slice_1 = epi_img_data[:, i, :]
            slice_2 = epi_img_data[:, :, i]

            slice_0 /= slice_0.max()/255.0
            slice_1 /= slice_1.max()/255.0
            slice_2 /= slice_2.max()/255.0

            fft_p0 = slice_0.astype(np.uint8)
            fft_p1 = slice_1.astype(np.uint8)
            fft_p2 = slice_2.astype(np.uint8)
            mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
            mri = Image.fromarray(mri)
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
            mri.save('fold4/train/{}'.format(name))        
 

    elif label=='AD':
        for i in range(46,51):
            slice_0 = epi_img_data[i, :, :]
            slice_1 = epi_img_data[:, i, :]
            slice_2 = epi_img_data[:, :, i]

            slice_0 /= slice_0.max()/255.0
            slice_1 /= slice_1.max()/255.0
            slice_2 /= slice_2.max()/255.0

            fft_p0 = slice_0.astype(np.uint8)
            fft_p1 = slice_1.astype(np.uint8)
            fft_p2 = slice_2.astype(np.uint8)

            mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
            mri = Image.fromarray(mri)
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
            mri.save('fold4/train/{}'.format(name))    
            

In [35]:
fold1_train={'File_name':train_file_names,
      'Label':train_labels}  
df_fold1_train= pd.DataFrame (fold1_train, columns =  ['File_name','Label'])
df_fold1_train.groupby('Label').count()
df_fold1_train
from sklearn.utils import shuffle
df_fold1_train = shuffle(df_fold1_train)
np.savetxt('train_fold4.txt', df_fold1_train.values,delimiter=" ", fmt="%s")
df_fold1_train.groupby('Label').count()

,File_name
Label,
AD,190
MCI,192
Normal,193


In [36]:
train5=pd.read_csv('dti_train_fold5.csv', index_col=0)
train5.groupby('Label').count()
train5

,DWI_file,day,Label
Subject,,,
OAS30076,FA_sub-OAS30076_ses-d1735_run-02_dwi.nii,1735,AD
OAS30085,FA_sub-OAS30085_ses-d1566_run-02_dwi.nii,1566,AD
OAS30136,FA_sub-OAS30136_ses-d0063_run-02_dwi.nii,63,AD
OAS30139,FA_sub-OAS30139_ses-d0101_run-02_dwi.nii,101,AD
OAS30206,FA_sub-OAS30206_ses-d2303_run-02_dwi.nii,2303,AD
...,...,...,...
OAS30324,FA_sub-OAS30324_ses-d0119_run-02_dwi.nii,119,Normal
OAS30326,FA_sub-OAS30326_ses-d0189_run-04_dwi.nii,189,Normal
OAS30328,FA_sub-OAS30328_ses-d0105_run-02_dwi.nii,105,Normal


In [37]:
train_labels=[]
train_file_names=[]
dftrain=train5
for i in range(len(dftrain['DWI_file'])):
    file_name=dftrain.iloc[i].DWI_file
    label=dftrain.iloc[i].Label

    mri = nib.load('/raid/aidana_massalimova/nii_data/FA/'+file_name)
    zooms = mri.header.get_zooms()
    old_affine = mri.affine
    new_affine = nib.affines.rescale_affine(affine=old_affine, shape=mri.get_fdata().shape, zooms=zooms, new_shape=(96,96,96))
    new_mri = image.resample_img(mri, target_affine=new_affine,target_shape=(96,96,96), interpolation='nearest')    
    epi_img_data = new_mri.get_fdata()
    if label=='Normal':
        slice_0 = epi_img_data[96//2, :, :]
        slice_1 = epi_img_data[:, 96//2, :]
        slice_2 = epi_img_data[:, :, 96//2]

        slice_0 /= slice_0.max()/255.0
        slice_1 /= slice_1.max()/255.0
        slice_2 /= slice_2.max()/255.0

        fft_p0 = slice_0.astype(np.uint8)
        fft_p1 = slice_1.astype(np.uint8)
        fft_p2 = slice_2.astype(np.uint8)
        mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
        mri = Image.fromarray(mri)
        name=file_name[0:-4]+'_1.png'
        train_labels.append(label)
        train_file_names.append(name)
        mri.save('fold5/train/{}'.format(name))                        
    elif label=='MCI':
        for i in range(24,72):
            slice_0 = epi_img_data[i, :, :]
            slice_1 = epi_img_data[:, i, :]
            slice_2 = epi_img_data[:, :, i]

            slice_0 /= slice_0.max()/255.0
            slice_1 /= slice_1.max()/255.0
            slice_2 /= slice_2.max()/255.0

            fft_p0 = slice_0.astype(np.uint8)
            fft_p1 = slice_1.astype(np.uint8)
            fft_p2 = slice_2.astype(np.uint8)
            mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
            mri = Image.fromarray(mri)
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
            mri.save('fold5/train/{}'.format(name))        
 

    elif label=='AD':
        for i in range(46,51):
            slice_0 = epi_img_data[i, :, :]
            slice_1 = epi_img_data[:, i, :]
            slice_2 = epi_img_data[:, :, i]

            slice_0 /= slice_0.max()/255.0
            slice_1 /= slice_1.max()/255.0
            slice_2 /= slice_2.max()/255.0

            fft_p0 = slice_0.astype(np.uint8)
            fft_p1 = slice_1.astype(np.uint8)
            fft_p2 = slice_2.astype(np.uint8)

            mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
            mri = Image.fromarray(mri)
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
            mri.save('fold5/train/{}'.format(name))    
            

In [38]:
fold1_train={'File_name':train_file_names,
      'Label':train_labels}  
df_fold1_train= pd.DataFrame (fold1_train, columns =  ['File_name','Label'])
df_fold1_train.groupby('Label').count()
df_fold1_train
from sklearn.utils import shuffle
df_fold1_train = shuffle(df_fold1_train)
np.savetxt('train_fold5.txt', df_fold1_train.values,delimiter=" ", fmt="%s")
df_fold1_train.groupby('Label').count()

,File_name
Label,
AD,190
MCI,192
Normal,193


In [39]:
test1=pd.read_csv('dti_test_fold1.csv', index_col=0)
test1.groupby('Label').count()


,DWI_file,day
Label,,
AD,10,10
MCI,1,1
Normal,48,48


In [40]:
train_labels=[]
train_file_names=[]
dftrain=test1
for i in range(len(dftrain['DWI_file'])):
    file_name=dftrain.iloc[i].DWI_file
    label=dftrain.iloc[i].Label

    mri = nib.load('/raid/aidana_massalimova/nii_data/FA/'+file_name)
    zooms = mri.header.get_zooms()
    old_affine = mri.affine
    new_affine = nib.affines.rescale_affine(affine=old_affine, shape=mri.get_fdata().shape, zooms=zooms, new_shape=(96,96,96))
    new_mri = image.resample_img(mri, target_affine=new_affine,target_shape=(96,96,96), interpolation='nearest')    
    epi_img_data = new_mri.get_fdata()
    if label=='Normal':
        slice_0 = epi_img_data[96//2, :, :]
        slice_1 = epi_img_data[:, 96//2, :]
        slice_2 = epi_img_data[:, :, 96//2]

        slice_0 /= slice_0.max()/255.0
        slice_1 /= slice_1.max()/255.0
        slice_2 /= slice_2.max()/255.0

        fft_p0 = slice_0.astype(np.uint8)
        fft_p1 = slice_1.astype(np.uint8)
        fft_p2 = slice_2.astype(np.uint8)
        mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
        mri = Image.fromarray(mri)
        name=file_name[0:-4]+'_1.png'
        train_labels.append(label)
        train_file_names.append(name)
        mri.save('fold1/test/{}'.format(name))                        
    elif label=='MCI':
        for i in range(24,72):
            slice_0 = epi_img_data[i, :, :]
            slice_1 = epi_img_data[:, i, :]
            slice_2 = epi_img_data[:, :, i]

            slice_0 /= slice_0.max()/255.0
            slice_1 /= slice_1.max()/255.0
            slice_2 /= slice_2.max()/255.0

            fft_p0 = slice_0.astype(np.uint8)
            fft_p1 = slice_1.astype(np.uint8)
            fft_p2 = slice_2.astype(np.uint8)
            mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
            mri = Image.fromarray(mri)
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
            mri.save('fold1/test/{}'.format(name))        
 

    elif label=='AD':
        for i in range(46,51):
            slice_0 = epi_img_data[i, :, :]
            slice_1 = epi_img_data[:, i, :]
            slice_2 = epi_img_data[:, :, i]

            slice_0 /= slice_0.max()/255.0
            slice_1 /= slice_1.max()/255.0
            slice_2 /= slice_2.max()/255.0

            fft_p0 = slice_0.astype(np.uint8)
            fft_p1 = slice_1.astype(np.uint8)
            fft_p2 = slice_2.astype(np.uint8)

            mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
            mri = Image.fromarray(mri)
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
            mri.save('fold1/test/{}'.format(name))    
            

In [41]:
fold1_train={'File_name':train_file_names,
      'Label':train_labels}  
df_fold1_train= pd.DataFrame (fold1_train, columns =  ['File_name','Label'])
df_fold1_train.groupby('Label').count()
df_fold1_train
from sklearn.utils import shuffle
df_fold1_train = shuffle(df_fold1_train)
np.savetxt('test_fold1.txt', df_fold1_train.values,delimiter=" ", fmt="%s")
df_fold1_train.groupby('Label').count()

,File_name
Label,
AD,50
MCI,48
Normal,48


In [42]:
test2=pd.read_csv('dti_test_fold2.csv', index_col=0)
test2.groupby('Label').count()


,DWI_file,day
Label,,
AD,10,10
MCI,1,1
Normal,48,48


In [43]:
train_labels=[]
train_file_names=[]
dftrain=test2
for i in range(len(dftrain['DWI_file'])):
    file_name=dftrain.iloc[i].DWI_file
    label=dftrain.iloc[i].Label

    mri = nib.load('/raid/aidana_massalimova/nii_data/FA/'+file_name)
    zooms = mri.header.get_zooms()
    old_affine = mri.affine
    new_affine = nib.affines.rescale_affine(affine=old_affine, shape=mri.get_fdata().shape, zooms=zooms, new_shape=(96,96,96))
    new_mri = image.resample_img(mri, target_affine=new_affine,target_shape=(96,96,96), interpolation='nearest')    
    epi_img_data = new_mri.get_fdata()
    if label=='Normal':
        slice_0 = epi_img_data[96//2, :, :]
        slice_1 = epi_img_data[:, 96//2, :]
        slice_2 = epi_img_data[:, :, 96//2]

        slice_0 /= slice_0.max()/255.0
        slice_1 /= slice_1.max()/255.0
        slice_2 /= slice_2.max()/255.0

        fft_p0 = slice_0.astype(np.uint8)
        fft_p1 = slice_1.astype(np.uint8)
        fft_p2 = slice_2.astype(np.uint8)
        mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
        mri = Image.fromarray(mri)
        name=file_name[0:-4]+'_1.png'
        train_labels.append(label)
        train_file_names.append(name)
        mri.save('fold2/test/{}'.format(name))                        
    elif label=='MCI':
        for i in range(24,72):
            slice_0 = epi_img_data[i, :, :]
            slice_1 = epi_img_data[:, i, :]
            slice_2 = epi_img_data[:, :, i]

            slice_0 /= slice_0.max()/255.0
            slice_1 /= slice_1.max()/255.0
            slice_2 /= slice_2.max()/255.0

            fft_p0 = slice_0.astype(np.uint8)
            fft_p1 = slice_1.astype(np.uint8)
            fft_p2 = slice_2.astype(np.uint8)
            mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
            mri = Image.fromarray(mri)
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
            mri.save('fold2/test/{}'.format(name))        
 

    elif label=='AD':
        for i in range(46,51):
            slice_0 = epi_img_data[i, :, :]
            slice_1 = epi_img_data[:, i, :]
            slice_2 = epi_img_data[:, :, i]

            slice_0 /= slice_0.max()/255.0
            slice_1 /= slice_1.max()/255.0
            slice_2 /= slice_2.max()/255.0

            fft_p0 = slice_0.astype(np.uint8)
            fft_p1 = slice_1.astype(np.uint8)
            fft_p2 = slice_2.astype(np.uint8)

            mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
            mri = Image.fromarray(mri)
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
            mri.save('fold2/test/{}'.format(name))    
            

In [44]:
fold1_train={'File_name':train_file_names,
      'Label':train_labels}  
df_fold1_train= pd.DataFrame (fold1_train, columns =  ['File_name','Label'])
df_fold1_train.groupby('Label').count()
df_fold1_train
from sklearn.utils import shuffle
df_fold1_train = shuffle(df_fold1_train)
np.savetxt('test_fold2.txt', df_fold1_train.values,delimiter=" ", fmt="%s")
df_fold1_train.groupby('Label').count()

,File_name
Label,
AD,50
MCI,48
Normal,48


In [45]:
test3=pd.read_csv('dti_test_fold3.csv', index_col=0)
test3.groupby('Label').count()


,DWI_file,day
Label,,
AD,9,9
MCI,1,1
Normal,49,49


In [46]:
train_labels=[]
train_file_names=[]
dftrain=test3
for i in range(len(dftrain['DWI_file'])):
    file_name=dftrain.iloc[i].DWI_file
    label=dftrain.iloc[i].Label

    mri = nib.load('/raid/aidana_massalimova/nii_data/FA/'+file_name)
    zooms = mri.header.get_zooms()
    old_affine = mri.affine
    new_affine = nib.affines.rescale_affine(affine=old_affine, shape=mri.get_fdata().shape, zooms=zooms, new_shape=(96,96,96))
    new_mri = image.resample_img(mri, target_affine=new_affine,target_shape=(96,96,96), interpolation='nearest')    
    epi_img_data = new_mri.get_fdata()
    if label=='Normal':
        slice_0 = epi_img_data[96//2, :, :]
        slice_1 = epi_img_data[:, 96//2, :]
        slice_2 = epi_img_data[:, :, 96//2]

        slice_0 /= slice_0.max()/255.0
        slice_1 /= slice_1.max()/255.0
        slice_2 /= slice_2.max()/255.0

        fft_p0 = slice_0.astype(np.uint8)
        fft_p1 = slice_1.astype(np.uint8)
        fft_p2 = slice_2.astype(np.uint8)
        mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
        mri = Image.fromarray(mri)
        name=file_name[0:-4]+'_1.png'
        train_labels.append(label)
        train_file_names.append(name)
        mri.save('fold3/test/{}'.format(name))                        
    elif label=='MCI':
        for i in range(24,72):
            slice_0 = epi_img_data[i, :, :]
            slice_1 = epi_img_data[:, i, :]
            slice_2 = epi_img_data[:, :, i]

            slice_0 /= slice_0.max()/255.0
            slice_1 /= slice_1.max()/255.0
            slice_2 /= slice_2.max()/255.0

            fft_p0 = slice_0.astype(np.uint8)
            fft_p1 = slice_1.astype(np.uint8)
            fft_p2 = slice_2.astype(np.uint8)
            mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
            mri = Image.fromarray(mri)
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
            mri.save('fold3/test/{}'.format(name))        
 

    elif label=='AD':
        for i in range(46,51):
            slice_0 = epi_img_data[i, :, :]
            slice_1 = epi_img_data[:, i, :]
            slice_2 = epi_img_data[:, :, i]

            slice_0 /= slice_0.max()/255.0
            slice_1 /= slice_1.max()/255.0
            slice_2 /= slice_2.max()/255.0

            fft_p0 = slice_0.astype(np.uint8)
            fft_p1 = slice_1.astype(np.uint8)
            fft_p2 = slice_2.astype(np.uint8)

            mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
            mri = Image.fromarray(mri)
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
            mri.save('fold3/test/{}'.format(name))    
            

In [47]:
fold1_train={'File_name':train_file_names,
      'Label':train_labels}  
df_fold1_train= pd.DataFrame (fold1_train, columns =  ['File_name','Label'])
df_fold1_train.groupby('Label').count()
df_fold1_train
from sklearn.utils import shuffle
df_fold1_train = shuffle(df_fold1_train)
np.savetxt('test_fold3.txt', df_fold1_train.values,delimiter=" ", fmt="%s")
df_fold1_train.groupby('Label').count()

,File_name
Label,
AD,45
MCI,48
Normal,49


In [48]:
test4=pd.read_csv('dti_test_fold4.csv', index_col=0)
test4.groupby('Label').count()


,DWI_file,day
Label,,
AD,9,9
MCI,1,1
Normal,48,48


In [49]:
train_labels=[]
train_file_names=[]
dftrain=test4
for i in range(len(dftrain['DWI_file'])):
    file_name=dftrain.iloc[i].DWI_file
    label=dftrain.iloc[i].Label

    mri = nib.load('/raid/aidana_massalimova/nii_data/FA/'+file_name)
    zooms = mri.header.get_zooms()
    old_affine = mri.affine
    new_affine = nib.affines.rescale_affine(affine=old_affine, shape=mri.get_fdata().shape, zooms=zooms, new_shape=(96,96,96))
    new_mri = image.resample_img(mri, target_affine=new_affine,target_shape=(96,96,96), interpolation='nearest')    
    epi_img_data = new_mri.get_fdata()
    if label=='Normal':
        slice_0 = epi_img_data[96//2, :, :]
        slice_1 = epi_img_data[:, 96//2, :]
        slice_2 = epi_img_data[:, :, 96//2]

        slice_0 /= slice_0.max()/255.0
        slice_1 /= slice_1.max()/255.0
        slice_2 /= slice_2.max()/255.0

        fft_p0 = slice_0.astype(np.uint8)
        fft_p1 = slice_1.astype(np.uint8)
        fft_p2 = slice_2.astype(np.uint8)
        mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
        mri = Image.fromarray(mri)
        name=file_name[0:-4]+'_1.png'
        train_labels.append(label)
        train_file_names.append(name)
        mri.save('fold4/test/{}'.format(name))                        
    elif label=='MCI':
        for i in range(24,72):
            slice_0 = epi_img_data[i, :, :]
            slice_1 = epi_img_data[:, i, :]
            slice_2 = epi_img_data[:, :, i]

            slice_0 /= slice_0.max()/255.0
            slice_1 /= slice_1.max()/255.0
            slice_2 /= slice_2.max()/255.0

            fft_p0 = slice_0.astype(np.uint8)
            fft_p1 = slice_1.astype(np.uint8)
            fft_p2 = slice_2.astype(np.uint8)
            mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
            mri = Image.fromarray(mri)
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
            mri.save('fold4/test/{}'.format(name))        
 

    elif label=='AD':
        for i in range(46,51):
            slice_0 = epi_img_data[i, :, :]
            slice_1 = epi_img_data[:, i, :]
            slice_2 = epi_img_data[:, :, i]

            slice_0 /= slice_0.max()/255.0
            slice_1 /= slice_1.max()/255.0
            slice_2 /= slice_2.max()/255.0

            fft_p0 = slice_0.astype(np.uint8)
            fft_p1 = slice_1.astype(np.uint8)
            fft_p2 = slice_2.astype(np.uint8)

            mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
            mri = Image.fromarray(mri)
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
            mri.save('fold4/test/{}'.format(name))    
            

In [50]:
fold1_train={'File_name':train_file_names,
      'Label':train_labels}  
df_fold1_train= pd.DataFrame (fold1_train, columns =  ['File_name','Label'])
df_fold1_train.groupby('Label').count()
df_fold1_train
from sklearn.utils import shuffle
df_fold1_train = shuffle(df_fold1_train)
np.savetxt('test_fold4.txt', df_fold1_train.values,delimiter=" ", fmt="%s")
df_fold1_train.groupby('Label').count()

,File_name
Label,
AD,45
MCI,48
Normal,48


In [51]:
test5=pd.read_csv('dti_test_fold5.csv', index_col=0)
test5.groupby('Label').count()


,DWI_file,day
Label,,
AD,9,9
MCI,1,1
Normal,48,48


In [52]:
train_labels=[]
train_file_names=[]
dftrain=test5
for i in range(len(dftrain['DWI_file'])):
    file_name=dftrain.iloc[i].DWI_file
    label=dftrain.iloc[i].Label

    mri = nib.load('/raid/aidana_massalimova/nii_data/FA/'+file_name)
    zooms = mri.header.get_zooms()
    old_affine = mri.affine
    new_affine = nib.affines.rescale_affine(affine=old_affine, shape=mri.get_fdata().shape, zooms=zooms, new_shape=(96,96,96))
    new_mri = image.resample_img(mri, target_affine=new_affine,target_shape=(96,96,96), interpolation='nearest')    
    epi_img_data = new_mri.get_fdata()
    if label=='Normal':
        slice_0 = epi_img_data[96//2, :, :]
        slice_1 = epi_img_data[:, 96//2, :]
        slice_2 = epi_img_data[:, :, 96//2]

        slice_0 /= slice_0.max()/255.0
        slice_1 /= slice_1.max()/255.0
        slice_2 /= slice_2.max()/255.0

        fft_p0 = slice_0.astype(np.uint8)
        fft_p1 = slice_1.astype(np.uint8)
        fft_p2 = slice_2.astype(np.uint8)
        mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
        mri = Image.fromarray(mri)
        name=file_name[0:-4]+'_1.png'
        train_labels.append(label)
        train_file_names.append(name)
        mri.save('fold5/test/{}'.format(name))                        
    elif label=='MCI':
        for i in range(24,72):
            slice_0 = epi_img_data[i, :, :]
            slice_1 = epi_img_data[:, i, :]
            slice_2 = epi_img_data[:, :, i]

            slice_0 /= slice_0.max()/255.0
            slice_1 /= slice_1.max()/255.0
            slice_2 /= slice_2.max()/255.0

            fft_p0 = slice_0.astype(np.uint8)
            fft_p1 = slice_1.astype(np.uint8)
            fft_p2 = slice_2.astype(np.uint8)
            mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
            mri = Image.fromarray(mri)
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
            mri.save('fold5/test/{}'.format(name))        
 

    elif label=='AD':
        for i in range(46,51):
            slice_0 = epi_img_data[i, :, :]
            slice_1 = epi_img_data[:, i, :]
            slice_2 = epi_img_data[:, :, i]

            slice_0 /= slice_0.max()/255.0
            slice_1 /= slice_1.max()/255.0
            slice_2 /= slice_2.max()/255.0

            fft_p0 = slice_0.astype(np.uint8)
            fft_p1 = slice_1.astype(np.uint8)
            fft_p2 = slice_2.astype(np.uint8)

            mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
            mri = Image.fromarray(mri)
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
            mri.save('fold5/test/{}'.format(name))    
            

In [ ]:
fold1_train={'File_name':train_file_names,
      'Label':train_labels}  
df_fold1_train= pd.DataFrame (fold1_train, columns =  ['File_name','Label'])
df_fold1_train.groupby('Label').count()
df_fold1_train
from sklearn.utils import shuffle
df_fold1_train = shuffle(df_fold1_train)
np.savetxt('test_fold5.txt', df_fold1_train.values,delimiter=" ", fmt="%s")
df_fold1_train.groupby('Label').count()